In [1]:

from pyspark.sql import SparkSession

# SparkSession = entry point for PySpark
spark = SparkSession.builder \
    .appName("PySpark_Basics") \
    .getOrCreate()

print("Spark Session Created Successfully")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/10 07:11:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Session Created Successfully


In [2]:
df = spark.read.csv("/kaggle/input/datasets/tawfikelmetwally/employee-dataset/Employee.csv", header=True, inferSchema=True)

In [4]:
df.show()

+---------+-----------+---------+-----------+---+------+-----------+-------------------------+----------+
|Education|JoiningYear|     City|PaymentTier|Age|Gender|EverBenched|ExperienceInCurrentDomain|LeaveOrNot|
+---------+-----------+---------+-----------+---+------+-----------+-------------------------+----------+
|Bachelors|       2017|Bangalore|          3| 34|  Male|         No|                        0|         0|
|Bachelors|       2013|     Pune|          1| 28|Female|         No|                        3|         1|
|Bachelors|       2014|New Delhi|          3| 38|Female|         No|                        2|         0|
|  Masters|       2016|Bangalore|          3| 27|  Male|         No|                        5|         1|
|  Masters|       2017|     Pune|          3| 24|  Male|        Yes|                        2|         1|
|Bachelors|       2016|Bangalore|          3| 22|  Male|         No|                        0|         0|
|Bachelors|       2015|New Delhi|          3| 

In [5]:
df.select("Education", "City").show(5)

+---------+---------+
|Education|     City|
+---------+---------+
|Bachelors|Bangalore|
|Bachelors|     Pune|
|Bachelors|New Delhi|
|  Masters|Bangalore|
|  Masters|     Pune|
+---------+---------+
only showing top 5 rows



In [7]:
df.filter(df.ExperienceInCurrentDomain > 5).show(5)

+---------+-----------+---------+-----------+---+------+-----------+-------------------------+----------+
|Education|JoiningYear|     City|PaymentTier|Age|Gender|EverBenched|ExperienceInCurrentDomain|LeaveOrNot|
+---------+-----------+---------+-----------+---+------+-----------+-------------------------+----------+
|Bachelors|       2016|Bangalore|          3| 39|  Male|         No|                        7|         0|
|Bachelors|       2016|     Pune|          3| 38|Female|         No|                        7|         0|
|Bachelors|       2016|Bangalore|          3| 33|  Male|         No|                        6|         0|
|Bachelors|       2014|     Pune|          3| 33|  Male|         No|                        6|         0|
|Bachelors|       2017|Bangalore|          3| 33|Female|         No|                        6|         0|
+---------+-----------+---------+-----------+---+------+-----------+-------------------------+----------+
only showing top 5 rows



In [8]:
df.groupBy("Gender").count().show()

+------+-----+
|Gender|count|
+------+-----+
|Female| 1875|
|  Male| 2778|
+------+-----+



In [10]:
df.orderBy("ExperienceInCurrentDomain", ascending=False).show(5)

+---------+-----------+---------+-----------+---+------+-----------+-------------------------+----------+
|Education|JoiningYear|     City|PaymentTier|Age|Gender|EverBenched|ExperienceInCurrentDomain|LeaveOrNot|
+---------+-----------+---------+-----------+---+------+-----------+-------------------------+----------+
|Bachelors|       2014|Bangalore|          3| 39|  Male|         No|                        7|         1|
|Bachelors|       2015|     Pune|          2| 41|Female|         No|                        7|         0|
|Bachelors|       2012|Bangalore|          3| 38|  Male|         No|                        7|         1|
|  Masters|       2014|Bangalore|          3| 40|Female|         No|                        7|         1|
|Bachelors|       2012|Bangalore|          1| 35|  Male|         No|                        7|         0|
+---------+-----------+---------+-----------+---+------+-----------+-------------------------+----------+
only showing top 5 rows



In [11]:
df.createOrReplaceTempView("employees")

In [13]:
spark.sql("""
SELECT Education, AVG(ExperienceInCurrentDomain) as avg_experience
FROM employees
GROUP BY Education
ORDER BY avg_experience DESC
""").show()

+---------+-----------------+
|Education|   avg_experience|
+---------+-----------------+
|      PHD|2.910614525139665|
|Bachelors|2.910580394334907|
|  Masters|2.884306987399771|
+---------+-----------------+



In [15]:
from pyspark.sql.functions import col

# New column based on condition
df = df.withColumn("high_experience", col("ExperienceInCurrentDomain") > 5)
df.show(5)

+---------+-----------+---------+-----------+---+------+-----------+-------------------------+----------+---------------+
|Education|JoiningYear|     City|PaymentTier|Age|Gender|EverBenched|ExperienceInCurrentDomain|LeaveOrNot|high_experience|
+---------+-----------+---------+-----------+---+------+-----------+-------------------------+----------+---------------+
|Bachelors|       2017|Bangalore|          3| 34|  Male|         No|                        0|         0|          false|
|Bachelors|       2013|     Pune|          1| 28|Female|         No|                        3|         1|          false|
|Bachelors|       2014|New Delhi|          3| 38|Female|         No|                        2|         0|          false|
|  Masters|       2016|Bangalore|          3| 27|  Male|         No|                        5|         1|          false|
|  Masters|       2017|     Pune|          3| 24|  Male|        Yes|                        2|         1|          false|
+---------+-----------+-

In [17]:
df_clean = df.dropna()

# Fill nulls
df_filled = df.fillna({"Education": 0})
df_filled.show(5)

+---------+-----------+---------+-----------+---+------+-----------+-------------------------+----------+---------------+
|Education|JoiningYear|     City|PaymentTier|Age|Gender|EverBenched|ExperienceInCurrentDomain|LeaveOrNot|high_experience|
+---------+-----------+---------+-----------+---+------+-----------+-------------------------+----------+---------------+
|Bachelors|       2017|Bangalore|          3| 34|  Male|         No|                        0|         0|          false|
|Bachelors|       2013|     Pune|          1| 28|Female|         No|                        3|         1|          false|
|Bachelors|       2014|New Delhi|          3| 38|Female|         No|                        2|         0|          false|
|  Masters|       2016|Bangalore|          3| 27|  Male|         No|                        5|         1|          false|
|  Masters|       2017|     Pune|          3| 24|  Male|        Yes|                        2|         1|          false|
+---------+-----------+-

In [18]:
df_clean.write.csv("clean_data.csv", header=True)

In [19]:
df_clean.write.parquet("clean_data.parquet")

In [20]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [23]:
assembler = VectorAssembler(inputCols=["ExperienceInCurrentDomain", "Age"], outputCol="features")
df_ml = assembler.transform(df_clean)

In [25]:
lr = LinearRegression(featuresCol="features", labelCol="ExperienceInCurrentDomain")

# 3️⃣ Fit model
model = lr.fit(df_ml)

# 4️⃣ Predictions
predictions = model.transform(df_ml)
predictions.select("ExperienceInCurrentDomain", "prediction").show(5)

26/02/10 07:23:29 WARN Instrumentation: [89edcc18] regParam is zero, which might cause numerical instability and overfitting.


+-------------------------+--------------------+
|ExperienceInCurrentDomain|          prediction|
+-------------------------+--------------------+
|                        0|-1.95695500404682...|
|                        3|                 3.0|
|                        2|  1.9999999999999991|
|                        5|   5.000000000000002|
|                        2|                 2.0|
+-------------------------+--------------------+
only showing top 5 rows

